In [1]:
import numpy as np
import h5py

In [5]:
from src.load_script import load_contest_train_dataset
from sklearn.model_selection import train_test_split

if True:
    X, y, samples = load_contest_train_dataset('datasets/contest_TRAIN.h5', 10)
    wavelengths = X.columns

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=samples)
    del X, y, samples
else:
    X_train = np.load(open('datasets/x_train.npy', 'rb'))
    y_train = np.load(open('datasets/y_train.npy', 'rb'))
    X_test = np.load(open('datasets/x_test.npy', 'rb'))
    y_test = np.load(open('datasets/y_test.npy', 'rb'))
    wavelengths = np.load(open('datasets/wavelengths.npy', 'rb'))

 58%|█████▊    | 7/12 [01:02<00:40,  8.16s/it]

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mutual_info_score

def euclid(a, b):
    return np.linalg.norm(a - b)

def cosine(a, b):
    return 1 - np.dot(a, b) / np.linalg.norm(a) / np.linalg.norm(b)

def direct_mutual_information(x, y, bins=100):
    c_xy, _, _ = np.histogram2d(x, y, bins)
    return - mutual_info_score(None, None, contingency=c_xy)

def mutual_information(a, b, bins=100):
    hgram, _, _ = np.histogram2d(a, b)
    pxy = hgram / float(np.sum(hgram))
    px = np.sum(pxy, axis=1)
    py = np.sum(pxy, axis=0)
    px_py = px[:, None] * py[None, :]
    nzs = pxy > 0
    return - np.sum(pxy[nzs] * np.log(pxy[nzs] / px_py[nzs]))


models = []
for metric in [euclid, cosine, mutual_information]: # cosine
    pipe = Pipeline([
        #('scaling', RobustScaler(unit_variance=True)),
        #('pca', PCA(whiten=False)),
        ('clf', KNeighborsClassifier(metric=metric, n_neighbors=5)),
    ])

    models.append(pipe.fit(X_train, y_train))

In [ ]:
from sklearn.metrics import classification_report
for model, model_name in zip(models, ['euclid', 'cosine', 'mutual_information']):
    print(model_name, sep=', best params were: ')
    print(classification_report(y_test, model.predict(X_test)))
    print('=' * 80)

euclid
              precision    recall  f1-score   support

           1       0.32      0.64      0.42        11
           2       0.34      0.58      0.43        19
           3       0.22      0.40      0.29         5
           4       0.78      0.88      0.82         8
           5       0.50      0.33      0.40         9
           6       1.00      0.80      0.89        10
           7       0.33      0.14      0.20         7
           8       0.67      0.36      0.47        11
           9       0.75      0.79      0.77        19
          10       0.83      0.83      0.83         6
          11       1.00      0.44      0.62        18
          12       1.00      0.33      0.50         9

    accuracy                           0.56       132
   macro avg       0.65      0.54      0.55       132
weighted avg       0.67      0.56      0.57       132

cosine
              precision    recall  f1-score   support

           1       0.48      0.91      0.62        11
          

c:\Users\Pavel\Desktop\git\libs-metrics\.venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Pavel\Desktop\git\libs-metrics\.venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Pavel\Desktop\git\libs-metrics\.venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r